In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Hàm tải dữ liệu từ thư mục ảnh
def load_data(data_dir):
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg"):
            label = filename.split("_")[0]
            if label in ["right", "forward", "left"]:
                img = cv2.imread(os.path.join(data_dir, filename))
                img = cv2.resize(img, (128, 128))  # Resize về kích thước 128x128
                images.append(img)
                labels.append(["right", "forward", "left"].index(label))
    return np.array(images), np.array(labels)

# Tải và chia dữ liệu
data_dir = "dataset"  # Thay bằng đường dẫn thực tế
images, labels = load_data(data_dir)
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

# Chuẩn hóa dữ liệu (giá trị pixel từ [0,255] về [0,1])
train_images = train_images / 255.0
val_images = val_images / 255.0

# Tăng cường dữ liệu để chống overfitting
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)

# Xây dựng mô hình CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),  # Dropout để chống overfitting
    layers.Dense(3, activation="softmax")  # 3 lớp: right, forward, left
])

# Compile mô hình
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=(val_images, val_labels),
    steps_per_epoch=len(train_images) // 32
)

# Vẽ đồ thị loss
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.savefig("cnn_loss_plot.png")
plt.close()

# Lưu mô hình dưới dạng .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("cnn_model.tflite", "wb") as f:
    f.write(tflite_model)


In [5]:
import os
from collections import Counter

# Đếm số lượng ảnh theo lớp
def count_labels(data_dir):
    label_counts = Counter()
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg"):
            label = filename.split("_")[0]
            if label in ["right", "forward", "left"]:
                label_counts[label] += 1
    return label_counts

# Gọi hàm và in kết quả
data_dir = "augmented_dataset"  # Thay bằng đường dẫn thư mục của bạn
counts = count_labels(data_dir)
print("Số lượng ảnh mỗi lớp:")
for label in ["forward", "left", "right"]:
    print(f"{label}: {counts[label]}")


Số lượng ảnh mỗi lớp:
forward: 723
left: 894
right: 723


In [ ]:
import os
import random

data_dir = "dataset"  # Thay bằng đường dẫn thực tế
label_to_reduce = "forward"
target_count = 192  # Số lượng mục tiêu để cân bằng

# Lọc ra danh sách file thuộc lớp 'forward'
all_files = [f for f in os.listdir(data_dir) if f.startswith(label_to_reduce) and f.endswith(".jpg")]

# Kiểm tra nếu số lượng ảnh vượt quá target_count thì mới xóa
if len(all_files) > target_count:
    files_to_delete = random.sample(all_files, len(all_files) - target_count)

    print(f"Đang xóa {len(files_to_delete)} ảnh lớp '{label_to_reduce}' để cân bằng...")
    for file in files_to_delete:
        file_path = os.path.join(data_dir, file)
        os.remove(file_path)
        print(f"Đã xóa: {file}")
else:
    print(f"Lớp '{label_to_reduce}' đã có ít hơn hoặc bằng {target_count} ảnh, không cần xóa.")
